In [ ]:
# Step 1: Import Libraries
import pandas as pd
import xml.etree.ElementTree as ET
import os
import re
import xmltodict

def xml_to_dict(element):
    """Recursively converts an XML element and its children to a dictionary."""
    if len(element) == 0:
        return element.text
    return {child.tag: xml_to_dict(child) for child in element}

def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [ ]:
# Step 2: Load and Parse XML Data using pandas.read_xml
def load_performance_data(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()
    print("Loading PP data for: ", os.path.basename(file_path))

    # Extract date from filename
    date_match = re.search(r'\d{8}', os.path.basename(file_path))
    race_date = ""
    if date_match:
        race_date = date_match.group(0)

    # Extract track from filename
    track_name = os.path.basename(file_path).split('_')[0][-3:]
    
    # Extract each Race element within EntryRaceCard and convert to a dictionary
    races = []
    for race in root.findall('.//Race'):
        race_dict = xmltodict.parse(ET.tostring(race))['Race']
        race_id = f"{race_date}_{race_dict['RaceNumber']}_{track_name}"
        race_date = race_date
        for entry in race.findall('.//Starters'):
            entry_dict = xmltodict.parse(ET.tostring(entry))['Starters']
            entry_dict['race_id'] = race_id
            entry_dict['race_date'] = race_date
            entry_dict['horse_id'] = f"{entry_dict['Horse']['HorseName']}_{entry_dict['ProgramNumber']}"
            races.append(entry_dict)
        
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(races)

    return df

performance_path = "C:\\Users\\dylan\\OneDrive - Wayne State College\\Documents\\Equibase Data 2023\\2023 PPs\\Extracted PPs"
file_suffixes = ['PRM_USA.xml', 'CBY_USA.xml', 'FON_USA.xml']  # Add your suffixes here

# Load all past performance files
performance_data = pd.concat([load_performance_data(os.path.join(root, file)) 
                              for root, _, files in os.walk(performance_path) 
                              for file in files if file.endswith(tuple(file_suffixes))])

In [ ]:
def load_results_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    print("Loading res data for: ", os.path.basename(file_path))

    entries = []
    for race in root.findall('.//RACE'):
        race_date = re.search(r'\d{8}', os.path.basename(file_path)).group(0)
        race_number = race.get("NUMBER")
        track_name = os.path.basename(file_path).split('_')[0][:3].upper()

        for entry in race.findall('.//ENTRY'):
            horse_name = entry.find(".//NAME").text
            horse_number = entry.find(".//PROGRAM_NUM").text
            entry_data = {
                "horse_id": f"{horse_name}_{horse_number}",
                "race_id": f"{race_date}_{race_number}_{track_name}",
                "ProgramNumber": horse_number,
                "HorseName": horse_name,
                "Position": entry.find(".//POINT_OF_CALL[@WHICH='FINAL']").find('.//POSITION').text,
            }
            entries.append(entry_data)
    
    return entries

results_path = "C:\\Users\\dylan\\OneDrive - Wayne State College\\Documents\\Equibase Data 2023\\2023 Result Charts"
file_prefixes = ['prm', 'cby', 'fon']  # Add your suffixes here


# Load all results files
all_races = []
for file in os.listdir(results_path):
    if file.endswith('.xml') and file.startswith(tuple(file_prefixes)):
        file_path = os.path.join(results_path, file)
        all_races.extend(load_results_data(file_path))

results_data = pd.DataFrame(all_races)

In [ ]:
# Step 3: Combine Data

# Merge the DataFrames on RaceNumber and race_date
merged_data = pd.merge(performance_data, results_data, on=["race_id", "horse_id"], how='inner')

# Clean up old DataFrames
del performance_data
del results_data

In [ ]:
# Drop unnecessary columns if they exist
cols_to_drop = ['WagerText', 'ParsedWagerText', 'ProgramSelections', 'RaceName', 'TrackRecord', 'SimulcastFlag']
existing_cols_to_drop = [col for col in cols_to_drop if col in merged_data.columns]
merged_data.drop(columns=existing_cols_to_drop, inplace=True)

In [ ]:
# Step 4: Data Preprocessing
merged_data.ffill(inplace=True)  # Forward fill missing values

# Check for remaining missing values
print("Missing values after ffill:")
print(merged_data.isnull().sum())

In [ ]:
def unpack_columns(df):
    def ensure_unique_columns(df):
        # Ensure all column names are unique
        cols = pd.Series(df.columns)
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
        df.columns = cols
        return df

    # Find all columns that are type list
    list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()]
    
    while len(list_columns) > 0:
        for col in list_columns:
            # Unpack list columns into new columns with a unique prefix
            df = df.join(df[col].apply(pd.Series).add_prefix(col + '_'))
            df = df.drop(columns=[col])
        
        # Ensure unique column names
        df = ensure_unique_columns(df)
        
        # Find all columns that are type dict
        dict_columns = df.columns[df.applymap(lambda x: isinstance(x, dict)).any()]
        
        while len(dict_columns) > 0:
            for col in dict_columns:
                flattened_df = pd.json_normalize(df[col])
                
                # Rename the columns to include the original column name as a prefix
                flattened_df.columns = [f"{col}_{subcol}" for subcol in flattened_df.columns]
                
                # Drop the original dict column and concatenate the flattened columns
                df = pd.concat([df.drop(columns=[col]), flattened_df], axis=1)
            
            # Ensure unique column names
            df = ensure_unique_columns(df)
            
            # Update dict_columns after flattening
            dict_columns = df.columns[df.applymap(lambda x: isinstance(x, dict)).any()]
        
        # Update list_columns after unpacking
        list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()]
    
    return df

merged_data = unpack_columns(merged_data)

In [ ]:
# # Remove all columns with less than 1% distinct values
# threshold = 0.01
# for col in merged_data.columns:
#     count = merged_data[col].count()
#     if count == 0:
#         continue  # Skip columns with no values
#     if merged_data[col].nunique() / count < threshold:
#         merged_data.drop(columns=[col], inplace=True)

In [ ]:
# Remove all columns that are 100% empty
merged_data.dropna(axis=1, how='all', inplace=True)

In [ ]:
# Find all columns containing .FirstName .MiddleName .LastName and drop them
name_columns = [col for col in merged_data.columns if '.FirstName' in col or '.MiddleName' in col or '.LastName' in col]
merged_data.drop(columns=name_columns, inplace=True)
del name_columns

# Find all columns containing Sire.HorseName or Dam.HorseName and drop them
pedigree_columns = [col for col in merged_data.columns if 'Sire.HorseName' in col or 'Dam.HorseName' in col]
merged_data.drop(columns=pedigree_columns, inplace=True)

# Find all columns containing Sire.FoalingDate or Dam.FoalingDate and drop them
pedigree_columns = [col for col in merged_data.columns if 'Sire.FoalingDate' in col or 'Dam.FoalingDate' in col]
merged_data.drop(columns=pedigree_columns, inplace=True)
del pedigree_columns

# Find all columns containing Trainer.ExternalPartyId and drop them
trainer_columns = [col for col in merged_data.columns if 'Trainer.ExternalPartyId' in col]
merged_data.drop(columns=trainer_columns, inplace=True)
del trainer_columns

# Find all columns containing Distance.PublishedValue and drop them
distance_columns = [col for col in merged_data.columns if 'Distance.PublishedValue' in col]
merged_data.drop(columns=distance_columns, inplace=True)
del distance_columns

# Find all columns containing CompanyLine and drop them
company_columns = [col for col in merged_data.columns if 'CompanyLine' in col]
merged_data.drop(columns=company_columns, inplace=True)
del company_columns

# Find all columns containing Scratch and drop them
scratch_columns = [col for col in merged_data.columns if 'Scratch' in col]
merged_data.drop(columns=scratch_columns, inplace=True)
del scratch_columns

In [ ]:
# Drop list of columns if they exist
cols_to_drop = ['RacingOwnerSilks', 'SaddleClothColor']
existing_cols_to_drop = [col for col in cols_to_drop if col in merged_data.columns]
merged_data.drop(columns=existing_cols_to_drop, inplace=True)
del cols_to_drop

In [ ]:
# Convert all columns containing the word 'Odds' from fraction to decimal
for col in merged_data.columns:
    if 'Odds' in col:
        merged_data[col] = merged_data[col].apply(lambda x: eval(x.replace('/', '/')) if isinstance(x, str) else x)

In [ ]:
# Find all columns that are strings but could be converted to a number and convert them
for col in merged_data.columns:
    if merged_data[col].apply(lambda x: isinstance(x, str) and x.replace('.', '', 1).isdigit()).all():
        merged_data[col] = pd.to_numeric(merged_data[col])

In [ ]:
# Find all columns that are objects but could be converted to a number and convert them
for col in merged_data.columns:
    if merged_data[col].apply(lambda x: isinstance(x, object) and str(x).replace('.', '', 1).isdigit()).all():
        merged_data[col] = pd.to_numeric(merged_data[col])

In [ ]:
# Find all columns containing the word 'Date' and convert them to datetime
for col in merged_data.columns:
    if 'Date' in col:
        merged_data[col] = pd.to_datetime(merged_data[col], yearfirst=True, errors='coerce')

In [ ]:
# Find all columns that are objects or strings but could be converted to a decimal and convert them
for col in merged_data.columns:
    if merged_data[col].apply(lambda x: x is None or (isinstance(x, object) and re.match(r'\d+\.\d+', str(x)) is not None)).all():
        merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Assuming merged_data is your DataFrame and 'Position' is the target column
target = merged_data['Position']
features = merged_data.drop(columns=['Position'])

# Identify categorical and datetime columns
categorical_cols = features.select_dtypes(include=['object']).columns
datetime_cols = features.select_dtypes(include=['datetime64']).columns
numeric_cols = features.select_dtypes(include=['number']).columns

# Convert datetime columns to numeric features (e.g., year, month, day)
for col in datetime_cols:
    features[col + '_year'] = features[col].dt.year
    features[col + '_month'] = features[col].dt.month
    features[col + '_day'] = features[col].dt.day

# Drop the original datetime columns
features = features.drop(columns=datetime_cols)

# Fill None values in categorical columns with a placeholder (e.g., 'missing')
features[categorical_cols] = features[categorical_cols].fillna('missing')

# Create a column transformer with imputers and ordinal encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler(with_mean=False))
        ]), numeric_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ]), categorical_cols)
    ],
    remainder='passthrough'  # Keep the remaining columns as they are
)

# Transform and normalize the features
normalized_features = preprocessor.fit_transform(features)

# Check for any remaining NaN values and handle them
if np.isnan(normalized_features).any():
    # Option 1: Drop rows with NaN values
    # normalized_features = normalized_features[~np.isnan(normalized_features).any(axis=1)]
    
    # Option 2: Fill remaining NaN values with a specific value (e.g., 0)
    normalized_features = np.nan_to_num(normalized_features, nan=0.0)

# Model Selection
regressor = RandomForestRegressor(n_estimators=10, random_state=0, oob_score=True)

# Training
X_train, X_test, y_train, y_test = train_test_split(normalized_features, target, test_size=0.2, random_state=42)
regressor.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.show()